Goal : to make a python file.

In [1]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def calculate_image_seq_on_dicompyler(image_no):
    return 136-image_no + 1

In [3]:
def display_scan(filename_text):
    plt.imshow(pydicom.dcmread(filename_text).pixel_array, cmap=plt.cm.bone)

In [4]:
integervalue = 3
'{0:03d}'.format(integervalue)
# image_uids --> sop_instance_ids_of_136_scans
image_uids = [] # data: ids of all images in the dataset
#image_locations = [] # data : locations of all slices/images in the dataset
uid_to_filename_map = dict()
uid_to_location_map = dict()
filename = 'data/P1152/P1152_pCT_20140226_'

# making a list of UIDs of all images
# going through all the files
for i in range(1, 137):
    current_filename = filename + '{0:03d}'.format(i) + '.dcm'
    current_file = pydicom.dcmread(current_filename)
    current_uid = current_file.SOPInstanceUID
    current_location = current_file.SliceLocation
    image_uids.append(current_uid)
    uid_to_filename_map[current_uid] = current_filename
    uid_to_location_map[current_uid] = current_location # think about merging these 2 dicts
    #image_locations.append(current_file.SliceLocation)  # think about using strings vs integers
    # computational cost.

The goal here is to find : the contours and extract them.

In [5]:
# contour_file_for_all_136_scans --> cffas
cffas = pydicom.dcmread('data/P1152/P1152_RTStructure_1.dcm')

In [6]:
contours = cffas.ROIContourSequence[9].ContourSequence[6].ContourData

In [7]:
def find_correct_image(contoursequence):
    """
    Given an rtstruct.dicom files ROIContourSequence[9].ContourSequence[i],
    return the correct filename (using the UID)
    """
    return uid_to_filename_map[contoursequence.ContourImageSequence[0].ReferencedSOPInstanceUID]

In [ ]:
filename_associated_with_contour = find_correct_image(cffas.ROIContourSequence[9].ContourSequence[6])

In [ ]:
no_of_image_in_filename = int(filename_associated_with_contour[-7:-4])
print(no_of_image_in_filename)
calculate_image_seq_on_dicompyler(no_of_image_in_filename)

In [ ]:
file_of_slice_with_contours = pydicom.dcmread(find_correct_image(cffas.ROIContourSequence[9].ContourSequence[6]))
pixel_array = file_of_slice_with_contours.pixel_array

In [ ]:
pixel_spacing = file_of_slice_with_contours.PixelSpacing
x_gap = pixel_spacing[0]
y_gap = pixel_spacing[1]
image_position_starting_points = file_of_slice_with_contours.ImagePositionPatient
x_start = float(image_position_starting_points[0])
y_start = float(image_position_starting_points[1])

In [ ]:
x_coords_of_contours = [ float(number) for number in contours[0::3] ]

In [ ]:
y_coords_of_contours = [ float(number) for number in contours[1::3] ]

In [ ]:
x_pixel_nos = (np.array(x_coords_of_contours)+300)/x_gap

In [ ]:
y_pixel_nos = (np.array(y_coords_of_contours)+271)/y_gap

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(pixel_array, cmap='bone')
plt.plot(x_pixel_nos, y_pixel_nos, color='r')
plt.figure()

At this stage, I try to find the pixels that fall within the contours in x_pixel_nos and y_pixel_nos.

In [ ]:
import cv2

In [ ]:
test = np.zeros_like(pixel_array)

In [ ]:
test

In [ ]:
cv2.drawContours(cimg, contours, i, color=255, thickness=-1)

Make an array of Nx1x2 as the number of total contours.

In [ ]:
cv2.drawContours(cimg, contours, i, color=255, thickness=-1)

In [ ]:
cimg = np.zeros_like(img)